In [ ]:
import zipfile

#using this data set below
#https://kern.humdrum.org/cgi-bin/browse?l=essen%2Feuropa%2Fdeutschl
with zipfile.ZipFile('deutschl.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped')
#worked!

save_path = 'dataset_mel' # save final processed data
file_dataset = 'file_dataset'
seq_length = 64

In [ ]:
import os
import music21 as m21 #to manipulate the music symbol data



In [ ]:
correct_notes = [0.25, 0.5, 0.75 , 1.0, 1.5, 2, 3, 4] #notes change to quarter format
#not enough samples of wierd note types so remove those songs since would need alot more varied data




In [ ]:
def load_songs(data_path):
  #loop through files and load
  songs = []

  for path, subdirs, files in os.walk(data_path):  #loop through all files in subfolders
    for file in files:
      #krn is last 3 letters
      if file[-3:] == "krn":
        song = m21.converter.parse(os.path.join(path, file))
        songs.append(song)
  return songs


def correct_length_notes(song, correct_len):
  for note in song.flat.notesAndRests: #flattens song to a list of notes and rests
    if note.duration.quarterLength not in correct_len:
      return False
  return True #all notes in song are correct format


def transpose_key(song): # need key or estimate using M21
#find distance to correct key, then transpose
#key is at index 4 or use getkey function if can't find.
#data set has no keychanges so no need to worry
  values = song.getElementsByClass(m21.stream.Part)
  key = values[0].getElementsByClass(m21.stream.Measure)[0][4]


  if not isinstance(key , m21.key.Key): #not a key
    key = song.analyze('key') # function for estimating key

  if key.mode == 'major': #transpose to cmaj
    inter = m21.interval.Interval(key.tonic, m21.pitch.Pitch('C'))

  if key.mode == 'minor': # go to amin
    inter = m21.interval.Interval(key.tonic , m21.pitch.Pitch('A'))

  tran_song = song.transpose(inter)

  return tran_song


def to_time_series(song):
  time_series = []
  for note in song.flat.notesAndRests:
    #notes
    if isinstance(note, m21.note.Note): # if note is a Note type
      val = note.pitch.midi # changes to midi note


    elif isinstance(note, m21.note.Rest): # checks if note is a rest
      val = 'rest'

    length = note.duration.quarterLength / 0.25
    length = int(length)

    for i in range(length):
      if i == 0:
        time_series.append(val)
      else:
        time_series.append('_') # held note


  time_series = " ".join(map(str, time_series)) # turns all to string then merges
  return time_series









mypath = 'unzipped/essen/europa/deutschl/test'


In [ ]:
x = load_songs(mypath)
print(len(x)) # should be 12 if worked ,  check folder
# x[0].show('text')
x = x[0]
print('good duration -', correct_length_notes(x, correct_notes))
#test - there should be 12 - worked!!

trans = transpose_key(x)





12
good duration - True


In [ ]:

def getData(path_dataset):
  pass
  #load
  print('loadin')

  songs = load_songs(path_dataset)
  print(f'loaded {len(songs)} songs' )

  for i , song in enumerate(songs):
  #filter out songs with unusual note values - so that they can be used for network
    if not correct_length_notes(song, correct_notes):
      continue # skips that song

    song = transpose_key(song)
    print('key changed')

    time_series  = to_time_series(song)


    #save to text file
    save_file = os.path.join(save_path, str(i)) #directory and file name

    with open(save_file, 'w') as fp:
      fp.write(time_series)

def load(file_path):
  with open(file_path, 'r') as fp:
    song = fp.read()
  return song


def merge(data_path, file_data_path, seq_length):
  #put seperators to individual files then merge
  new_song_sep = "/ " * seq_length
  songs = ""

  for path, subdirs, files in os.walk(data_path):
    for file in files:
      file_path = os.path.join(path, file)
      song = load(file_path)
      songs = songs + song + " " + new_song_sep #string of all songs

  songs = songs[:-1] #gets rid of last empty space

  with open(file_data_path, 'w') as fp:
    fp.write(songs) # whole string of all songs

  return songs




In [ ]:
#first make a folder with the name of mypath that you want to use!!!!
getData(mypath) #worked for first set of 12


loadin
loaded 12 songs
key changed
key changed
key changed
key changed
key changed
key changed
key changed
key changed
key changed
key changed
key changed
key changed


In [ ]:
songs = merge(save_path, file_dataset, seq_length)
print(songs)
#worked!

55 _ _ _ 64 _ _ _ 64 _ 64 _ 64 _ _ _ 62 _ 62 _ 62 _ _ _ _ _ 60 _ 60 _ _ _ 55 _ _ _ 65 _ _ _ 65 _ 65 _ 65 _ 64 _ 67 _ 64 _ 64 _ _ _ 62 _ _ _ rest _ _ _ 55 _ _ _ 67 _ _ _ 67 _ 67 _ 67 _ _ _ 65 _ 65 _ 64 _ _ _ _ _ _ _ 67 _ 65 _ 64 _ 62 _ 60 _ _ _ 60 _ 60 _ 64 _ 62 _ 62 _ _ _ 60 _ _ _ _ _ _ _ 62 _ _ _ 65 _ _ _ 64 _ _ _ 64 _ 64 _ 67 _ 65 _ 64 _ 62 _ 60 _ _ _ _ _ _ _ rest _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 64 _ 69 _ _ _ 71 _ 72 _ _ 71 69 _ 76 _ _ _ _ _ 71 _ _ _ 74 _ 72 _ _ 71 69 _ 68 _ _ _ 69 _ 71 _ _ _ _ _ rest _ _ _ 64 _ 76 _ _ _ 76 _ 72 _ _ 71 69 _ 81 _ _ _ _ _ 77 _ _ _ 74 _ 72 _ _ 71 69 _ 71 _ _ 69 68 _ 69 _ _ _ _ _ rest _ _ _ / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / / 76 _ _ _ _ _ 74 _ 72 _ 64 _ 69 _ _ _ 67 _ 65 _ 62 _ _ _ 69 _ _ _ 67 _ 65 _ 62 _ _ _ 69 _ _ _ 67 _ 64 _ 60 _ _ rest 76 _ _ _ _ _ 74 _ 72 _ 64

In [ ]:
import json
import tensorflow.keras as keras
import numpy as np

In [ ]:
def mapper(songs, tab_path):
  dict_tab = {}

  songs = songs.split() # splits all vals
  outs = list(set(songs)) #set removes dupes

  for i , value in enumerate(outs):
    dict_tab[value] = i

  with open(tab_path , "w") as fp:
    # pickle.dump - can cause security issues since can run arbitrary code. so don't use
    json.dump(dict_tab, fp, indent = 4)


  return dict_tab

tab_path = 'tab_dict.json'
dict_tab = mapper(songs, tab_path)
print(dict_tab)

#the nums are midi notes

{'74': 0, '69': 1, 'rest': 2, '62': 3, '55': 4, '/': 5, '76': 6, '_': 7, '77': 8, '57': 9, '64': 10, '60': 11, '67': 12, '68': 13, '65': 14, '71': 15, '81': 16, '72': 17}


In [ ]:
#convert songs to intergers for network
def convert(songs):
  songs_nums = []

  with open(tab_path , 'r') as fp:
    convert = json.load(fp)

  songs = songs.split()

  for note in songs:
    songs_nums.append(convert[note])

  return songs_nums

def train_samps(sequen_length):
  #[1,2,3] -> [1,2] target is 3  -the next value we want to predict
  #map to integers
  inputs = []
  target_val = []

  songs = load(file_dataset)
  songs_num = convert(songs)

  num_samps = len(songs_num) - sequen_length

  for i in range(num_samps):
    inputs.append(songs_num[i:i+sequen_length])
    target_val.append(songs_num[i+sequen_length])

  #inputs shape -> #num of seq, seq len , span of midi values
  span = len(set(songs_num))
  inputs = keras.utils.to_categorical(inputs , num_classes=span)
  target_val = np.array(target_val)


  return inputs, target_val





In [ ]:
inputs, target_vals = train_samps(seq_length)
print(inputs.shape)
print(target_vals.shape)


(2512, 64, 18)
(2512,)
